In [15]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
# from scipy.misc import imread, imresize (depreciated)

In [6]:
# Import the sys module, which provides access to some variables used or maintained by the Python interpreter
import sys

# Install the imageio package using pip
# The '!{sys.executable}' part ensures that pip installs the package in the same Python environment as the current Jupyter kernel
!{sys.executable} -m pip install imageio

  Obtaining dependency information for imageio from https://files.pythonhosted.org/packages/c0/69/3aaa69cb0748e33e644fda114c9abd3186ce369edd4fca11107e9f39c6a7/imageio-2.33.1-py3-none-any.whl.metadata
Using cached imageio-2.33.1-py3-none-any.whl (313 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Chang.LAPTOP-KLP71L1N\anaconda3\python.exe -m pip install --upgrade pip


In [7]:
import imageio
from PIL import Image

### Saving the model weights

In [17]:
def process_image(image):
    # Read the image using imageio
    img = imageio.imread(image)

    # Resize the image using PIL
    img = Image.fromarray(img)
    img = img.resize((256, 256)) #256x256x3

    # Convert the image to a numpy array and transpose to make channels first
    img = np.array(img).transpose(2, 0, 1)

    # Normalize pixel values to [0, 1]
    img = img / 255.

    # Convert the numpy array to a PyTorch tensor and move it to GPU
    img = torch.FloatTensor(img).cpu()

    # Define a normalization transform
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Apply the normalization transform
    img = normalize(img) # (3, 256, 256)

    return img

In [10]:
# Define the model:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # First convolutional layer: 3 input channels, 6 output channels, 5x5 kernel, with default 0 padding and stride of 1
        self.conv1 = nn.Conv2d(3, 6, 5)   
        
        # Max pooling layer with 2x2 kernel and stride 2
        self.pool = nn.MaxPool2d(2, 2)  
        
        # Second convolutional layer: 6 input channels, 12 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 12, 5)  
        
        # First fully connected layer: input size is 12*61*61, output size is 120
        self.fc1 = nn.Linear(12 * 61 * 61, 120)
        
        # Second fully connected layer: input size is 120, output size is 10
        self.fc2 = nn.Linear(120, 10)

    # consider an image of size 256x256:
    def forward(self, x):
        # Forward pass of the input through the CNN:
        
        # output size = [ (256 - 5 + 2(0) ) / 1 ] + 1 --> 252x252:
        x = F.relu(self.conv1(x))                # Apply first convolutional layer and ReLU
        
        # output_size = 252 / 2 --> 126x126:
        x = self.pool(x)                         # Apply max pooling
        
        # output size = [ (126 - 5 + 2(0) ) / 1 ] + 1 --> 122x122:
        x = F.relu(self.conv2(x))                # Apply second convolutional layer and ReLU
        
        # output size = 122/2 --> 61x61:
        x = self.pool(x)                         # Apply max pooling
        
        # (1, 44652):
        x = x.view(-1, 12 * 61 * 61)             # Flatten the output for the fully connected layer
        
        # (1, 120):
        x = F.relu(self.fc1(x))                  # Apply first fully connected layer and ReLU
        
        # (1, 10):
        x = self.fc2(x)                          # Apply second fully connected layer
        
        return x

In [13]:
# Initialize the CNN model
model = CNN()

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Initialize the optimizer with Stochastic Gradient Descent (SGD)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [18]:
# Process the image using the defined function
image = process_image('test.jpg')

# Add a batch dimension to the image tensor
image = image.unsqueeze(0)  # This changes the shape from [C, H, W] to [1, C, H, W]

C:\Users\Chang.LAPTOP-KLP71L1N\AppData\Local\Temp\ipykernel_17340\4175269825.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image)


In [20]:
# Feed the processed image through the model
output = model(image)

# Print the shape of the output
print(output.shape)

torch.Size([1, 10])


In [21]:
# Print model's state_dict:

print("Model's state_dict:")

for param_tensor in model.state_dict():
    # Iterate through each parameter tensor in the state dictionary
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([12, 6, 5, 5])
conv2.bias 	 torch.Size([12])
fc1.weight 	 torch.Size([120, 44652])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([10, 120])
fc2.bias 	 torch.Size([10])


In [22]:
# Save the model's state dictionary to a file
torch.save(model.state_dict(), 'model.pth.tar')

## After Saving Model State Dictionary

In [23]:
# Uncomment the following line if you need to create a new instance of the model:
# model = CNN()

# Load the saved state dictionary into the model
model.load_state_dict(torch.load('model.pth.tar'))

model.eval()     #set dropout and batch normalization layers to evaluation mode before inference (testing)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

## Saving & Loading a General Checkpoint for Inference and/or to Resuming Training

In [24]:
# Create a new instance of the CNN model and move it to the CPU:
model = CNN().cpu()

In [25]:
# Initialize the SGD optimizer:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [26]:
# Create a checkpoint dictionary:
checkpoint = {
    'epoch': 1,  # The current epoch number
    'model_state_dict': model.state_dict(),  # The state dictionary of the model (weights, biases, etc.)
    'optimizer_state_dict': optimizer.state_dict(),  # The state dictionary of the optimizer (momentum, learning rate, etc.)
    'loss': 0.2  # The loss value at the time of saving
}

# Save the checkpoint to a file
torch.save(checkpoint, 'model.pth.tar')

In [27]:
# Load Checkpoint:
checkpoint = torch.load('model.pth.tar')

In [28]:
# Load the saved model state into the model
model.load_state_dict(checkpoint['model_state_dict'])

# Load the saved optimizer state into the optimizer
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Retrieve the epoch number from the checkpoint
epoch = checkpoint['epoch']

# Retrieve the loss value from the checkpoint
loss = checkpoint['loss']

In [29]:
# If testing
model.eval()

# If resuming training
model.train()

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)